In [93]:
import datetime, time
a = "1527266992000"
b = "1527266998000"
print(datetime.datetime.fromtimestamp(int(a)/1000).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.datetime.fromtimestamp(int(b)/1000).strftime('%Y-%m-%d %H:%M:%S'))

2018-05-26 00:49:52
2018-05-26 00:49:58


In [14]:
from scapy.all import *
import struct

class TcpStream():
    def __init__(self, pkt):
        self.init = True
        if 'Raw' in pkt:
            self.data = pkt[Raw].load
#             self.data = b''
            self.nseq = pkt[TCP].seq + len(self.data)
#             print('---> ', self.nseq, pkt[TCP].seq, )
        else:
            self.data = b''
            if pkt[TCP].flags & 2 == 2:
                self.nseq = pkt[TCP].seq  + 1
    def add_packet(self, pkt):
        if pkt[TCP].seq != self.nseq:
            print(pkt[TCP].seq, self.nseq)
            print('Oops')
            return
        if 'Raw' in pkt:
            self.data += pkt[Raw].load
            self.nseq = pkt[TCP].seq + len(pkt[Raw].load)
            if self.init:
                if pkt[Raw].load[:2] == b'ES':
                    self.init = False
                    self.c = len(self.data) - len(pkt[Raw].load)
#             print('******> ', self.data)
            if not self.init:
#                 print('**********************************************')
                for i in range(5):
                    found_frame = False
#                     print (len(self.data) - self.c - 1, len(self.data), self.c)
                    if len(self.data) - self.c >= 6:
#                         print('1111111111111111111111111111111111')
                        magic, size = struct.unpack('>hi', self.data[self.c:self.c+6])
                        if size == -1: ## ping header
                            self.data = self.data[self.c+6:]
                            self.c = 0
                        else:
                            if len(self.data) - self.c >= 19:
                                magic, size, requestId, status, version_id = struct.unpack('>hiqbi', self.data[self.c:self.c+19])
                                print(size, magic, size, requestId, status, version_id)
                                if len(self.data) - self.c >= size + 6:
                                    content = self.data[self.c+19:self.c+19+size-13]
#                                     print('=====> ',content)
                                    if status == 0:
                                        print('############> ', decodeRequest(content))
                                        reqs[requestId] = pkt.time
                                        if requestId == 141386056:
                                            print('%%%%%%%%%%%%%%%> ', content)
                                    if status == 1:
                                        try:
                                            t = pkt.time - reqs[requestId]
                                            print('----------> request[%s] use %f sec' % (requestId, t))
                                        except:
                                            pass
                                    self.data = self.data[self.c+19+size-13:]
                                    self.c = 0
                    if not found_frame:
#                         print('##################')
                        break
#                     else:
#                         print('##################')
#                         break
#             print('====>', pkt[TCP].seq, len(pkt[Raw].load))
        else:
#             self.data = ''
            if pkt[TCP].flags & 3 : # SYN or FIN
                self.nseq = pkt[TCP].seq  + 1
    
streams = {}
reqs = {}
def deal_pkt(p):
    if (p[TCP].sport, p[TCP].dport) not in streams:
        if p[TCP].flags == 16:
            pass
        else:
            s = TcpStream(p)
            streams[(p[TCP].sport, p[TCP].dport)] = s
    else:
        s = streams[(p[TCP].sport, p[TCP].dport)]
        s.add_packet(p)

In [52]:
import pysmile,json

def readVInt(buf):
#     print(buf[:10])
    ret = 0
    for i in range(5):
        a = buf[i]
        ret += (a&0x7f) << (i*7)
        first = a & 0x80
        if first == 0:
            return ret, buf[i+1:]

def readStr(buf):
    str_len, buf = readVInt(buf)
    s = buf[:str_len]
    return s.decode('utf-8'), buf[str_len:]

def deal_bin(buf):
    ret = bytearray(len(buf))
    for i in range(len(buf)):
        first_bit = buf[i] & 0x80
        if first_bit:
            ret[i] = ord('^')
        else:
            ret[i] = buf[i]
    return ret

def decodeRequest(buf):
    ret = {}
    s, buf = readStr(buf)
    ret['action'] = s
    if s == 'indices:data/read/search':
        ret.update(decodeSearchRequest(buf))
    elif s == 'search':
        ret.update(decodeSearchRequest_11(buf))
    elif s == 'indices:data/read/search[phase/query]':
        ret.update(decodeShardSearchRequest(buf))
    else:
        ret['buff'] = buf
    return ret

def decodeSearchRequest(buf):
    ret = {}
    header_flag = buf[0]
    assert(header_flag == 0)
    buf = buf[1:]
    search_type = buf[0]
    buf = buf[1:]
    indices_count, buf = readVInt(buf)
    indices = []
    ret['indices'] = indices
#     print(buf)
    for i in range(indices_count):
        s, buf = readStr(buf)
        indices.append(s)
    print('**> ', indices)
    if buf[0]:
        s, buf = readStr(buf[1:])
        ret['routing'] = s
    else:
        buf = buf[1:]
    if buf[0]:
        s, buf = readStr(buf[1:])
        ret['preference'] = s
    else:
        buf = buf[1:]
    scroll_flag = buf[0]
#     assert(scroll_flag == 0)
    buf = buf[1:]
    source_len, buf = readVInt(buf)
    source_bin, buf = buf[:source_len], buf[source_len]
    print(source_len)
    print(source_bin)
    ret['source'] = json.dumps(pysmile.decode(source_bin), ensure_ascii=False)
#     ret['source_2'] = deal_bin(source_bin)
    return ret

def decodeSearchRequest_11(buf):
    ret = {}
    header_flag = buf[0]
    assert(header_flag == 0)
    buf = buf[2:]
    search_type = buf[0]
    buf = buf[1:]
    indices_count, buf = readVInt(buf)
    indices = []
    ret['indices'] = indices
#     print(buf)
    for i in range(indices_count):
        s, buf = readStr(buf)
        indices.append(s)
    print('**> ', indices)
    if buf[0]:
        s, buf = readStr(buf[1:])
        ret['routing'] = s
    else:
        buf = buf[1:]
    if buf[0]:
        s, buf = readStr(buf[1:])
        ret['preference'] = s
    else:
        buf = buf[1:]
    scroll_flag = buf[0]
#     assert(scroll_flag == 0)
    buf = buf[1:]
    source_len, buf = readVInt(buf)
    source_bin, buf = buf[:source_len], buf[source_len]
    print(source_len)
    print(source_bin)
    ret['source'] = json.dumps(pysmile.decode(source_bin), ensure_ascii=False)
#     ret['source_2'] = deal_bin(source_bin)
    return ret

# ShardSearchTransportRequest.java
def decodeShardSearchRequest(buf):
    ret = {}
    header_flag = buf[0]
    assert(header_flag == 0)
    buf = buf[1:]
    s, buf = readStr(buf)
    ret['index'] = s
    shardId, buf = readVInt(buf)
    ret['shardId'] = shardId
    searchType = buf[0]
    buf = buf[1:]
    numberOfShard, buf = readVInt(buf)
#     print(s, shardId, numberOfShard)
    scroll_flag = buf[0]
    assert(scroll_flag == 0)
    buf = buf[1:]
    source_len, buf = readVInt(buf)
#     print(source_len, buf)
    source_bin, buf = buf[:source_len], buf[source_len]
#     print(source_len)
#     print(source_bin)
    ret['source'] = json.dumps(pysmile.decode(source_bin), ensure_ascii=False)
    return ret

In [41]:
bytes('北京','utf-8')
readVInt(b'\x97')

IndexError: index out of range

In [8]:
json.dumps({'name':'王巍'}, ensure_ascii=False)

'{"name": "王巍"}'

In [50]:
a = {'action': 'search', 'buff': b'\x00\x02\x00\x01\ra_mt_ves_read\x00\x01\npreference\x00\x99\x03:)\n\x05\xfa\x83from\xc0\x83size$\xa8\x84query\xfa\x83bool\xfa\x83must\xf8\xfa\x83term\xfa\x87deviceIdS35021176001320000026\xfb\xfb\xfa\x84range\xfa\x88q_network\xfa@\xc0\x81to\xc2\x8cinclude_lower#\x8cinclude_upper#\xfb\xfb\xfb\xfaG\xfa\x88q_shelter\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x87q_freeze\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x85q_blur\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x90q_brightness_high\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x8fq_brightness_low\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x84q_hue\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x8dq_interference\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x86q_shake\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x88q_sigloss\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xf9\xfb\xfb\x86explain"\x83sort\xf8\xfa\x83date\xfa\x84orderCdesc\xfb\xfb\xf9\xfb\x00\x01\x08a_mt_ves\x06\x00\x00\x01\n\x00'}
b = b'ES\x00\x00\x00\xe2\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x10T\x13\x18indices:data/read/search\x00\x01\x01\x12a_haiou_hit_result\x00\x00\x00\x97\x01:)\n\x05\xfa\x84query\xfa\x87filtered\xfa@\xfa\x84range\xfa\x8asearch_time\xfa\x83from!\x81to!\x8cinclude_lower#\x8cinclude_upper#\xfb\xfb\xfb\x85filter\xfaB\xfaC\xfaDR2017-03-07 13:42:25ER2017-03-08 13:42:25F#G#\xfb\xfb\xfb\xfb\xfb\xfb\x00\x00&\x00\x00\x00\x01\n\x00\x02'
print(a)
print(b)
# a['buff'] = b'\x00\x02\x02\x01\x12a_haiou_hit_result\x00\x00\x00E:)\n\x05\xfa\x84query\xfa\x87filtered\xfa@\xfa\x83term\xfa\x82abc\x84\xe5\x8c\x97\xe4\xba\xac\xfb\xfb\x85filter\xfaB\xfa\x85abcdefBshg\xfb\xfb\xfb\xfb\xfb\x00\x00\x06\x00\x00\x01\n\x00'

{'action': 'search', 'buff': b'\x00\x02\x00\x01\ra_mt_ves_read\x00\x01\npreference\x00\x99\x03:)\n\x05\xfa\x83from\xc0\x83size$\xa8\x84query\xfa\x83bool\xfa\x83must\xf8\xfa\x83term\xfa\x87deviceIdS35021176001320000026\xfb\xfb\xfa\x84range\xfa\x88q_network\xfa@\xc0\x81to\xc2\x8cinclude_lower#\x8cinclude_upper#\xfb\xfb\xfb\xfaG\xfa\x88q_shelter\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x87q_freeze\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x85q_blur\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x90q_brightness_high\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x8fq_brightness_low\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x84q_hue\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x8dq_interference\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x86q_shake\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x88q_sigloss\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xf9\xfb\xfb\x86explain"\x83sort\xf8\xfa\x83date\xfa\x84orderCdesc\xfb\xfb\xf9\xfb\x00\x01\x08a_mt_ves\x06\x00\x00\x01\n\x00'}
b'ES\x00\x00\x00\xe2\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x10T\x13\x18

In [36]:
b = 'preference'
len(b)
# ord('\r')

10

In [51]:
decodeSearchRequest(a['buff'])

**>  ['a_mt_ves_read']
409
b':)\n\x05\xfa\x83from\xc0\x83size$\xa8\x84query\xfa\x83bool\xfa\x83must\xf8\xfa\x83term\xfa\x87deviceIdS35021176001320000026\xfb\xfb\xfa\x84range\xfa\x88q_network\xfa@\xc0\x81to\xc2\x8cinclude_lower#\x8cinclude_upper#\xfb\xfb\xfb\xfaG\xfa\x88q_shelter\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x87q_freeze\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x85q_blur\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x90q_brightness_high\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x8fq_brightness_low\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x84q_hue\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x8dq_interference\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x86q_shake\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xfaG\xfa\x88q_sigloss\xfa@\xc0I\xc2J#K#\xfb\xfb\xfb\xf9\xfb\xfb\x86explain"\x83sort\xf8\xfa\x83date\xfa\x84orderCdesc\xfb\xfb\xf9\xfb'


{'indices': ['a_mt_ves_read'],
 'preference': 'preference',
 'source': '{"from": 0, "size": 20, "explain": false, "sort": [{"date": {"order": "desc"}}], "query": {"bool": {"must": [{"term": {"deviceId": "35021176001320000026"}}, {"range": {"q_network": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_shelter": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_freeze": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_blur": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_brightness_high": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_brightness_low": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_hue": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range": {"q_interference": {"from": 0, "include_lower": true, "to": 1, "include_upper": true}}}, {"range

In [50]:
'%x' % 132
0x84 >> 5
ba = bytearray(b'd')
ba.decode('utf-8')
c = 'ooo'
a = {1:'a'}
b = {2:'b'}
a.update(b)
a

{1: 'a', 2: 'b'}

In [19]:
# import pysmile
c = ':)\n\x05\xfa\x84query\xfa\x87filtered\xfa@\xfa\x84range\xfa\x8asearch_time\xfa\x83from!\x81to!\x8cinclude_lower#\x8cinclude_upper#\xfb\xfb\xfb\x85filter\xfaB\xfaC\xfaDR2017-03-07 13:42:25ER2017-03-08 13:42:25F#G#\xfb\xfb\xfb\xfb\xfb\xfb'
len(c)
b = b'\x18indices:data/read/search\x00\x01\x01\x12a_haiou_hit_result\x00\x00\x00B:)\n\x05\xfa\x84query\xfa\x87filtered\xfa@\xfa\x83term\xfa\x82abc\x84\xe5\x8c\x97\xe4\xba\xac\xfb\xfb\x85filter\xfaB\xfaC\x84\xe4\xb8\x8a\xe6\xb5\xb7\xfb\xfb\xfb\xfb\xfb\x00\x00&\x00\x00\x00\x01\n\x00\x02'
b = b'\x12cluster/nodes/info\x00\x01\x06_local\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
readVInt(b)
decodeRequest(b)

('cluster/nodes/info',
 b'\x00\x01\x06_local\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [43]:
# open('pysmile/decode.py', 'rb').read()

In [35]:
pkts = rdpcap('/tmp/zushou/t3.pcap')
p = pkts[0]
for i in pkts:
    print(repr(i))

<Ether  dst=a4:bf:01:16:e1:3e src=40:55:39:21:7d:c1 type=0x800 |<IP  version=4 ihl=5 tos=0x0 len=97 id=15936 flags=DF frag=0 ttl=127 proto=tcp chksum=0x97d3 src=172.22.14.28 dst=192.168.170.168 options=[] |<TCP  sport=51810 dport=9300 seq=4200255021 ack=3752307139 dataofs=5 reserved=0 flags=PA window=32624 chksum=0x53fd urgptr=0 options=[] |<Raw  load='ES\x00\x00\x003\x00\x00\x00\x00\x00*\xa1\xd0\x00\x00\x10T\x13\x15cluster:monitor/state\x00\x00\x00\x00\x06\xfc#\xac\x00\x00\x01\x00\x00\x00\x01\x07' |>>>>
<Ether  dst=a4:bf:01:16:e1:3e src=40:55:39:21:7d:c1 type=0x800 |<IP  version=4 ihl=5 tos=0x0 len=46 id=16029 flags=DF frag=0 ttl=127 proto=tcp chksum=0x97a9 src=172.22.14.28 dst=192.168.170.168 options=[] |<TCP  sport=51810 dport=9300 seq=4200255078 ack=3752308043 dataofs=5 reserved=0 flags=PA window=32850 chksum=0x8031 urgptr=0 options=[] |<Raw  load='ES\xff\xff\xff\xff' |>>>>
<Ether  dst=a4:bf:01:16:e1:3e src=40:55:39:21:7d:c1 type=0x800 |<IP  version=4 ihl=5 tos=0x0 len=97 id=16142 

In [55]:
streams = {}
# sniff(prn=deal_pkt, offline='/tmp/zushou/t3.pcap', filter='src port 51810')
# sniff(prn=deal_pkt, offline='/tmp/hadoop4_2.pcap', filter='dst port 44926')
sniff(prn=deal_pkt, offline='/tmp/112.pcap', filter='')
# sniff(prn=deal_pkt, offline='/tmp/suzhou.pcap')
# sniff(prn=deal_pkt, offline='/tmp/guangxi.pcap')


51 17747 51 0 0 1010299
############>  {'action': 'cluster/nodes/info', 'buff': b'\x00\x01\x06_local\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01'}
7833 17747 7833 0 1 1010299
7833 17747 7833 0 1 1010299
7833 17747 7833 0 1 1010299
7833 17747 7833 0 1 1010299
----------> request[0] use 0.000779 sec
124 17747 124 1 0 1010299
**>  ['a_haiou_hit_result']
69
b':)\n\x05\xfa\x84query\xfa\x87filtered\xfa@\xfa\x83term\xfa\x82abc\x84\xe5\x8c\x97\xe4\xba\xac\xfb\xfb\x85filter\xfaB\xfa\x85abcdefBshg\xfb\xfb\xfb\xfb\xfb'
############>  {'source': '{"query": {"filtered": {"filter": {"term": {"abcdef": "shg"}}, "query": {"term": {"abc": "北京"}}}}}', 'action': 'search', 'indices': ['a_haiou_hit_result']}
29 17747 29 1 1 1010299
----------> request[1] use 0.001496 sec


<Sniffed: TCP:69 UDP:0 ICMP:0 Other:0>

In [10]:
al = open('/tmp/dump/4_2/035.048.097.039.09300-035.048.097.043.44256', 'rb').read()

In [15]:
magic, size, requestId, status, version_id = struct.unpack('>hiqbi', al[:19])
magic, size, requestId, status, version_id

(17747, 37, 163618325, 1, 1070099)